<a href="https://colab.research.google.com/github/Devi-Prasad-G/Drug_target_interaction/blob/main/deep_cca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

#creating embeddings

In [ ]:
def create_dataset(x1_embeds,x2,path):
  with open(path,'r') as f:
    x1_molecules = f.readlines()
  #print(len(x1_molecules),x1_molecules[0])
  x1_molecules = map(lambda s: s.strip(), x1_molecules) #removing \n from all strings
  x1 = []
  for i in x1_molecules:
    new_val = np.mean(x1_embeds[i],axis = 0)
    #print(new_val.shape)
    x1.append(new_val)
  return np.array(x1),np.squeeze(x2)

In [ ]:
train_x1_embeds = np.load("/content/drive/MyDrive/drug_target_interaction/compounds.npz")
train_x2 = np.load("/content/drive/MyDrive/drug_target_interaction/train_target.npy")
train_path = "/content/drive/MyDrive/drug_target_interaction/compounds.txt"
train_x1,train_x2 = create_dataset(train_x1_embeds,train_x2,train_path)
print(train_x1.shape,train_x2.shape)

(36592, 512) (36592, 1024)


In [ ]:
val_x1_embeds = np.load("/content/drive/MyDrive/drug_target_interaction/val_compounds.npz")
val_x2 = np.load("/content/drive/MyDrive/drug_target_interaction/val_target.npy")
val_path = "/content/drive/MyDrive/drug_target_interaction/val_compounds.txt"
val_x1,val_x2 = create_dataset(val_x1_embeds,val_x2,val_path)
print(val_x1.shape,val_x2.shape)

(5227, 512) (5227, 1024)


In [ ]:
test_x1_embeds = np.load("/content/drive/MyDrive/drug_target_interaction/test_compounds.npz")
test_x2 = np.load("/content/drive/MyDrive/drug_target_interaction/test_target.npy")
test_path = "/content/drive/MyDrive/drug_target_interaction/test_compounds.txt"
test_x1,test_x2 = create_dataset(test_x1_embeds,test_x2,test_path)
print(test_x1.shape,test_x2.shape)

(10455, 512) (10455, 1024)


In [ ]:
path =  "/content/drive/MyDrive/drug_target_interaction/"
train_y,val_y,test_y = np.load(path+"train_y.npy"),np.load(path+"val_y.npy"),np.load(path+"test_y.npy")

In [ ]:
data1 = np.array([(train_x1, train_x2),(val_x1, val_x2),(test_x1, test_x2)],dtype=object)

In [ ]:
np.save(path+"data1.npy",data1,allow_pickle=True)

In [ ]:
print(path)

/content/drive/MyDrive/drug_target_interaction/


In [ ]:
train_set_x1, train_set_x2 = data1[0]
valid_set_x1, valid_set_x2 = data1[1]
test_set_x1, test_set_x2 = data1[2]

In [ ]:
print(train_set_x1.shape, train_set_x2.shape)

(36592, 512) (36592, 1024)


In [ ]:
data2 = np.array([(train_x1, train_y),(val_x1, val_y),(test_x1, test_y)],dtype=object)
data3 = np.array([(train_x2, train_y),(val_x2, val_y),(test_x2, test_y)],dtype=object)

In [ ]:
np.save(path+"data2.npy",data2,allow_pickle=True)
np.save(path+"data3.npy",data3,allow_pickle=True)

#loading data

In [ ]:
path =  "/content/drive/MyDrive/drug_target_interaction/"
data1 = np.load(path+"data1.npy",allow_pickle = True)
data2 = np.load(path+"data2.npy",allow_pickle = True)
data3 = np.load(path+"data3.npy",allow_pickle = True)

#deep CCA model

In [ ]:
!pip install tensorflow~=2.5.0
#import tensorflow as tf

In [ ]:
import os

import numpy as np
#from keras.callbacks import ModelCheckpoint

from linear_cca import linear_cca
from models import create_model
#from utils import load_data, svm_classify
import tensorflow as tf

In [ ]:
def train_model(model, data1, epoch_num: int, batch_size: int):
    """
    trains the model
    # Arguments
        data1 and data2: the train, validation, and test data for view 1 and view 2 respectively. data should be packed
        like ((X for train, Y for train), (X for validation, Y for validation), (X for test, Y for test))
        epoch_num: number of epochs to train the model
        batch_size: the size of batches
    # Returns
        the trained model
    """

    # Unpacking the data
    train_set_x1, train_set_x2 = data1[0]
    valid_set_x1, valid_set_x2 = data1[1]
    test_set_x1, test_set_x2 = data1[2]

    # best weights are saved in "temp_weights.hdf5" during training
    # it is done to return the best model based on the validation loss
    #checkpointer = ModelCheckpoint(
    #    filepath=os.path.join(os.getcwd(), "temp_weights.h5"),
    #    verbose=1,
    #    save_best_only=True,
    #    save_weights_only=True,
    #)

    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath="/content/drive/MyDrive/drug_target_interaction/deep_cca.h5",
        monitor='val_loss',
        mode = 'min',
        save_best_only=True)
    # used dummy Y because labels are not used in the loss function
    model.fit(
        x=[train_set_x1, train_set_x2],
        y=np.zeros(train_set_x1.shape[0]),
        batch_size=batch_size,
        epochs=epoch_num,
        shuffle=True,
        validation_data=([valid_set_x1, valid_set_x2], np.zeros(valid_set_x1.shape[0])),
        callbacks=[checkpoint],
    )

    #model.load_weights(os.path.join(os.getcwd(), "examples", "temp_weights.h5"))

    results = model.evaluate(
        x=[test_set_x1, test_set_x2],
        y=np.zeros(test_set_x1.shape[0]),
        batch_size=batch_size,
        verbose=1,
    )

    print("loss on test data: ", results)

    results = model.evaluate(
        x=[valid_set_x1, valid_set_x2],
        y=np.zeros(valid_set_x1.shape[0]),
        batch_size=batch_size,
        verbose=1,
    )
    print("loss on validation data: ", results)
    return model

In [ ]:
def test_model(model, data1, data2, outdim_size: int, apply_linear_cca: bool):
    """produce the new features by using the trained model
    # Arguments
        model: the trained model
        data1 and data2: the train, validation, and test data for view 1 and view 2 respectively.
            Data should be packed like
            ((X for train, Y for train), (X for validation, Y for validation), (X for test, Y for test))
        outdim_size: dimension of new features
        apply_linear_cca: if to apply linear CCA on the new features
    # Returns
        new features packed like
            ((new X for train - view 1, new X for train - view 2, Y for train),
            (new X for validation - view 1, new X for validation - view 2, Y for validation),
            (new X for test - view 1, new X for test - view 2, Y for test))
    """

    # producing the new features
    new_data = []
    for k in range(3):
        pred_out = model.predict([data1[k][0], data2[k][0]])
        r = int(pred_out.shape[1] / 2)
        new_data.append([pred_out[:, :r], pred_out[:, r:], data1[k][1]])

    # based on the DCCA paper, a linear CCA should be applied on the output of the networks because
    # the loss function actually estimates the correlation when a linear CCA is applied to the output of the networks
    # however it does not improve the performance significantly
    if apply_linear_cca:
        w = [None, None]
        m = [None, None]
        print("Linear CCA started!")
        w[0], w[1], m[0], m[1] = linear_cca(new_data[0][0], new_data[0][1], outdim_size)
        print("Linear CCA ended!")

        # Something done in the original MATLAB implementation of DCCA, do not know exactly why;)
        # it did not affect the performance significantly on the noisy MNIST dataset
        # s = np.sign(w[0][0,:])
        # s = s.reshape([1, -1]).repeat(w[0].shape[0], axis=0)
        # w[0] = w[0] * s
        # w[1] = w[1] * s
        ###

        for k in range(3):
            data_num = len(new_data[k][0])
            for v in range(2):
                new_data[k][v] -= m[v].reshape([1, -1]).repeat(data_num, axis=0)
                new_data[k][v] = np.dot(new_data[k][v], w[v])

    return new_data

In [ ]:
# Parameters Section

# the size of the new space learned by the model (number of the new features)
outdim_size = 64

# size of the input for view 1 and view 2
input_shape1 = 512
input_shape2 = 1024

# number of layers with nodes in each one
layer_sizes1 = [512, 128, outdim_size]
layer_sizes2 = [1024, 512, 128, outdim_size]

# the parameters for training the network
learning_rate = 1e-3
epoch_num = 100
batch_size = 128

# the regularization parameter of the network
# seems necessary to avoid the gradient exploding especially when non-saturating activations are used
reg_par = 1e-5

# specifies if all the singular values should get used to calculate the correlation or just the top outdim_size ones
# if one option does not work for a network or dataset, try the other one
use_all_singular_values = True

# if a linear CCA should get applied on the learned features extracted from the networks
# it does not affect the performance on noisy MNIST significantly
apply_linear_cca = True

# end of parameters section
############

# Building, training, and producing the new features by DCCA
model = create_model(
    layer_sizes1,
    layer_sizes2,
    input_shape1,
    input_shape2,
    learning_rate,
    reg_par,
    outdim_size,
    use_all_singular_values,
)
#model.build(input_shape=(512,))
model.summary()

#data1 = np.load("/content/drive/MyDrive/drug_target_interaction/data1.npy", allow_pickle=True)

#same data as data1 but stored in different format

#data2 = np.load("/content/drive/MyDrive/drug_target_interaction/data2.npy", allow_pickle=True)
#data3 = np.load("/content/drive/MyDrive/drug_target_interaction/data3.npy", allow_pickle=True)

model = train_model(model, data1, epoch_num, batch_size)
new_data = np.array(test_model(model, data2, data3, outdim_size, apply_linear_cca), dtype="object")
# Training and testing of SVM with linear kernel on the view 1 with new features
#[test_acc, valid_acc] = svm_classify(new_data, C=0.01)
#print("Accuracy on view 1 (validation data) is:", valid_acc * 100.0)
#print("Accuracy on view 1 (test data) is:", test_acc * 100.0)

# Saving new features in a gzip pickled file specified by save_to
#print("saving new features ...")
# the path to save the final learned features
#save_to = os.path.join(os.getcwd(), "new_features.npy")
#np.save(save_to, new_data)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 view2_input (InputLayer)       [(None, 1024)]       0           []                               
                                                                                                  
 view1_input (InputLayer)       [(None, 512)]        0           []                               
                                                                                                  
 dense_3 (Dense)                (None, 1024)         1049600     ['view2_input[0][0]']            
                                                                                                  
 dense (Dense)                  (None, 512)          262656      ['view1_input[0][0]']            
                                                                                              

In [ ]:
print(new_data.shape)

(3, 3)


In [ ]:
np.save("/content/drive/MyDrive/drug_target_interaction/new_data.npy",new_data,allow_pickle = True)

#loading new data from drive

In [ ]:
new_data = np.load("/content/drive/MyDrive/drug_target_interaction/new_data.npy",allow_pickle = True)

In [ ]:
new_train_x1,new_train_x2,new_train_y = new_data[0][0],new_data[0][1], new_data[0][2]
new_val_x1,new_val_x2,new_val_y = new_data[1][0],new_data[1][1], new_data[1][2]
new_test_x1,new_test_x2,new_test_y = new_data[2][0],new_data[2][1], new_data[2][2]

(5227, 64)


In [ ]:
new_data[0][1].shape

(36592, 64)